In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch
!pip install -r requirements.txt

/content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 34.3 MB/s 
     |████████████████████████████████| 592 kB 68.6 MB/s 
     |████████████████████████████████| 2.3 MB 61.7 MB/s 
     |████████████████████████████████| 3.1 MB 62.0 MB/s 
     |████████████████████████████████| 408 kB 73.7 MB/s 
     |████████████████████████████████| 3.5 MB 62.1 MB/s 
     |████████████████████████████████| 34.6 MB 85.4 MB/s 
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5709 sha256=968bfb9191f3e0763692a46da28f810ceafdb83132525d02a94e822bf6ce1045
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built torchfile
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfull

In [ ]:
%cd /content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch

import torch
import torch.nn as nn
import torch.optim as optim
from models.utils import preprocess_data
import numpy as np
import torchvision
import cv2
from torchvision import datasets, models, transforms
import sys
import numpy
from models.vgg_face import get_pretrained_model, get_prediction
from image_utils import save_image, show_image  
import torch.nn.functional as F
import logging
import math

numpy.set_printoptions(threshold=sys.maxsize)


logger = logging.getLogger("benignAccuracy-logger")

logging.basicConfig(
    level=logging.INFO,
    filename="logs/benignAccuracy.log",
    filemode="w",
    format="%(asctime)s %(name)s %(levelname)s: %(message)s"
)

names = [line.rstrip('\n') for line in open('data/names.txt')]
 
BATCH_SIZE = 100

model = get_pretrained_model()

model.eval()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

dataloader, dataset_size = preprocess_data(BATCH_SIZE)

correct = 0
total = 0
cnt = 1

for data in dataloader:
  print(f"Durchlauf: {cnt}/{math.ceil(dataset_size / BATCH_SIZE)}")

  images, labels = data
  images = images[:,[2,1,0],:,:] #rgb to bgr

  images = images.to(device)
  labels = labels.to(device)

  values, indices = get_prediction(images, model)

  predictions = [(f"id: {index}", f"name: {names[index]}", f"confidence: {value}") for index, value in zip(indices.tolist(), values.tolist())]
  ground_truth = [(f"id: {label}", f"name: {names[label]}") for label in labels.tolist()]   

  num_pred = labels.size(0)
  pred_correct = (indices == labels).sum().item()

  correct += pred_correct
  total += num_pred

  logger.info(f"\nBatch {cnt}/{math.ceil(dataset_size / BATCH_SIZE)} - Predictions: {predictions}\n"\
              f"Batch {cnt}/{math.ceil(dataset_size / BATCH_SIZE)} - Labels: {ground_truth}\n"\
              f"Batch {cnt}/{math.ceil(dataset_size / BATCH_SIZE)} - Num. Predictions: {num_pred}\n"\
              f"Batch {cnt}/{math.ceil(dataset_size / BATCH_SIZE)} - Correct Predictions: {pred_correct}")

  cnt += 1

accuracy = correct / total 

logger.info(f"\nNum. correct predictions: {correct}\n"\
            f"Total num. predictions: {total}\n"\
            f"Accuracy: {accuracy}")

/content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch
['aaron_staton', 'adam_copeland', 'amitabh_bachchan', 'andrew_rannells', 'ben_whishaw', 'chris_riggi', 'connie_nielsen', 'elaine_hendrix', 'eli_roth', 'jeannie_mai', 'jennifer_ferrin', 'john_mahoney', 'josh_lucas', 'katherine_jenkins', 'kathryn_mccormick', 'kelly_rowan', 'kiersten_warren', 'lana_wachowski', 'lauren_bowles', 'lennie_james', 'marshall_allman', 'michael_gambon', 'moran_atias', 'ralf_little', 'salli_richardson-whitfield', 'tommy_morrison', 'tracy_morgan', 'virginia_williams', 'wagner_moura', 'zac_efron']
Dataset size: 900 entries
Durchlauf: 1/9
Durchlauf: 2/9
Durchlauf: 3/9
Durchlauf: 4/9
Durchlauf: 5/9
Durchlauf: 6/9
Durchlauf: 7/9
Durchlauf: 8/9
Durchlauf: 9/9
